In [1]:
import os
import torch
import GPUtil
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [2]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear/non_linear_1000.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/non_linear')
MODEL_RESULTS_AE_PATH = os.path.join(ROOT_PATH, r'results/non_linear/model_autoencoder_AE')
MODEL_RESULTS_PGNNIV_PATH = os.path.join(ROOT_PATH, r'results/non_linear/model_autoencoder_NN')


# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_AE_PATH)
create_folder(MODEL_RESULTS_PGNNIV_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/model_autoencoder_AE
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/model_autoencoder_NN


In [3]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear/non_linear_1000.pkl


In [4]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [5]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


### División de los datos

In [6]:
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
y_train = torch.Tensor(dataset['y_train']).unsqueeze(1)
K_train = torch.tensor(dataset['k_train']).unsqueeze(1)
f_train = torch.tensor(dataset['f_train']).unsqueeze(1).to(torch.float32)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1)
y_val = TensOps(torch.Tensor(dataset['y_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val']).to(torch.float32).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

print("Train dataset length:", len(X_train))
print("Validation dataset length:", len(X_val))

Train dataset length: 800
Validation dataset length: 200


In [7]:
N_data_AE = len(X_train)//2
N_data_NN = len(X_train) - len(X_train)//2
prop_data_NN = 1 - N_data_AE/(N_data_NN + N_data_AE)

print("Dataset length for the autoencoder:", N_data_AE)
print("Dataset length for the PGNNIV:", N_data_NN)

X_AE, X_NN, y_AE, y_NN, K_AE, K_NN, f_AE, f_NN = train_test_split(X_train, y_train, K_train, f_train, test_size=prop_data_NN, random_state=42)

Dataset length for the autoencoder: 400
Dataset length for the PGNNIV: 400


#### Datos para el autoencoder

In [8]:
y_train_AE, y_test_AE = train_test_split(y_AE, test_size=0.2, random_state=42)

y_train_AE = TensOps(y_train_AE.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
y_test_AE = TensOps(y_test_AE.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

#### Datos para la PGNNIV

In [9]:
X_train_NN, X_test_NN, y_train_NN, y_test_NN, K_train_NN, K_test_NN, f_train_NN, f_test_NN = train_test_split(X_NN, y_NN, K_NN, f_NN, test_size=0.2, random_state=42)

X_train_NN = X_train_NN.to(DEVICE)
X_test_NN = X_test_NN.to(DEVICE)

y_train_NN = TensOps(y_train_NN.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
y_test_NN = TensOps(y_test_NN.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

K_train_NN = TensOps(K_train_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
K_test_NN = TensOps(K_test_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

f_train_NN = TensOps(f_train_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
f_test_NN = TensOps(f_test_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

## Autoencoder

In [10]:
from models import Autoencoder
from trainers.train import train_autoencoder_loop

In [ ]:
autoencoder_input_shape = y_train_AE.values[0].shape
latent_space_dim = [15, 10, 3, 10, 15]
autoencoder_output_shape = y_train_AE.values[0].shape

In [12]:
X_train = y_train_AE.values
y_train = y_train_AE

X_test = y_test_AE.values
y_test = y_test_AE

In [13]:
autoencoder = Autoencoder(autoencoder_input_shape, latent_space_dim, autoencoder_output_shape).to(DEVICE)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-2)

start_epoch = 0
n_epochs = 10000
batch_size = 64
n_checkpoint = 10
new_lr = None

train_autoencoder_loop(autoencoder, optimizer, X_train, y_train, X_test, y_test,  
                       n_checkpoint, start_epoch, n_epochs, batch_size, MODEL_RESULTS_AE_PATH, DEVICE, new_lr)

Starting training from scratch.
Epoch 0, Train loss: 5.316e+01, Test loss: 4.328e+01
Epoch 100, Train loss: 5.224e-01, Test loss: 5.029e-01
Epoch 200, Train loss: 4.676e-01, Test loss: 5.330e-01
Epoch 300, Train loss: 4.573e-01, Test loss: 4.865e-01
Epoch 400, Train loss: 4.501e-01, Test loss: 4.735e-01
Epoch 500, Train loss: 4.443e-01, Test loss: 4.681e-01
Epoch 600, Train loss: 4.475e-01, Test loss: 4.629e-01
Epoch 700, Train loss: 4.427e-01, Test loss: 4.597e-01
Epoch 800, Train loss: 4.459e-01, Test loss: 4.713e-01
Epoch 900, Train loss: 4.469e-01, Test loss: 4.594e-01
Epoch 1000, Train loss: 4.502e-01, Test loss: 4.634e-01
Epoch 1100, Train loss: 4.549e-01, Test loss: 4.643e-01
Epoch 1200, Train loss: 4.556e-01, Test loss: 4.628e-01
Epoch 1300, Train loss: 4.590e-01, Test loss: 4.663e-01
Epoch 1400, Train loss: 4.487e-01, Test loss: 4.631e-01
Epoch 1500, Train loss: 4.463e-01, Test loss: 4.684e-01
Epoch 1600, Train loss: 4.475e-01, Test loss: 4.638e-01
Epoch 1700, Train loss: 4.46

In [14]:
start_epoch = 9000
n_epochs = 20000
batch_size = 64
n_checkpoint = 10
new_lr = 1e-4

train_autoencoder_loop(autoencoder, optimizer, X_train, y_train, X_test, y_test,  
                       n_checkpoint, start_epoch, n_epochs, batch_size, MODEL_RESULTS_AE_PATH, DEVICE, new_lr)

Starting training from a checkpoint. Epoch 9000.
Epoch 9000, Train loss: 2.644e-03, Test loss: 2.650e-03
Epoch 9100, Train loss: 5.754e-04, Test loss: 1.011e-03
Epoch 9200, Train loss: 5.651e-04, Test loss: 9.925e-04
Epoch 9300, Train loss: 5.541e-04, Test loss: 9.728e-04
Epoch 9400, Train loss: 5.429e-04, Test loss: 9.531e-04
Epoch 9500, Train loss: 5.321e-04, Test loss: 9.347e-04
Epoch 9600, Train loss: 5.222e-04, Test loss: 9.182e-04
Epoch 9700, Train loss: 5.133e-04, Test loss: 9.039e-04
Epoch 9800, Train loss: 5.055e-04, Test loss: 8.916e-04
Epoch 9900, Train loss: 4.984e-04, Test loss: 8.809e-04
Epoch 10000, Train loss: 4.916e-04, Test loss: 8.713e-04
Epoch 10100, Train loss: 4.846e-04, Test loss: 8.623e-04
Epoch 10200, Train loss: 4.771e-04, Test loss: 8.533e-04
Epoch 10300, Train loss: 4.689e-04, Test loss: 8.440e-04
Epoch 10400, Train loss: 4.601e-04, Test loss: 8.342e-04
Epoch 10500, Train loss: 4.510e-04, Test loss: 8.241e-04
Epoch 10600, Train loss: 4.420e-04, Test loss: 8.

## PGNNIV

In [15]:
from vecopsciml.operators.zero_order import Mx, My
from models.ae_nonlinear_model import AutoencoderNonlinearModel

In [16]:
# Predictive network architecture
input_shape = X_train_NN[0].shape
predictive_layers = [15, 10, 3]
predictive_output = y_train_NN.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train_NN)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train_NN)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [17]:
pretrained_decoder = autoencoder.decoder

for param in pretrained_decoder.parameters():
    param.requires_grad = False

for name, param in pretrained_decoder.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

hidden1_layer.weight: requires_grad=False
hidden1_layer.bias: requires_grad=False
hidden2_layer.weight: requires_grad=False
hidden2_layer.bias: requires_grad=False
output_layer.weight: requires_grad=False
output_layer.bias: requires_grad=False


In [18]:
model = AutoencoderNonlinearModel(input_shape, predictive_layers, pretrained_decoder, predictive_output, explanatory_input,
                                   explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 10000

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train_NN, y_train_NN, f_train_NN, X_test_NN, y_test_NN, f_test_NN,
           D, n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PGNNIV_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 1.533e+08, Test loss: 1.442e+08, MSE(e): 7.498e+00, MSE(pi1): 7.584e+03, MSE(pi2): 4.479e+00, MSE(pi3): 2.480e+01
Epoch 100, Train loss: 2.688e+06, Test loss: 3.126e+06, MSE(e): 2.668e-01, MSE(pi1): 5.695e-01, MSE(pi2): 1.865e-01, MSE(pi3): 1.351e-01
Epoch 200, Train loss: 2.257e+05, Test loss: 5.338e+05, MSE(e): 2.147e-02, MSE(pi1): 2.141e-01, MSE(pi2): 1.313e-02, MSE(pi3): 8.846e-02
Epoch 300, Train loss: 1.203e+05, Test loss: 3.343e+05, MSE(e): 1.108e-02, MSE(pi1): 2.000e-01, MSE(pi2): 6.981e-03, MSE(pi3): 7.525e-02
Epoch 400, Train loss: 9.473e+04, Test loss: 2.508e+05, MSE(e): 8.648e-03, MSE(pi1): 1.870e-01, MSE(pi2): 5.323e-03, MSE(pi3): 6.373e-02
Epoch 500, Train loss: 8.483e+04, Test loss: 1.927e+05, MSE(e): 7.759e-03, MSE(pi1): 1.851e-01, MSE(pi2): 4.582e-03, MSE(pi3): 5.381e-02
Epoch 600, Train loss: 6.673e+04, Test loss: 1.714e+05, MSE(e): 6.069e-03, MSE(pi1): 1.617e-01, MSE(pi2): 3.392e-03, MSE(pi3): 4.413e-02
Epoch 700, 

In [19]:
# Parametros de entrenamiento
start_epoch = 9000
n_epochs = 100000

batch_size = 64 
n_checkpoints = 100

second_lr = 1e-4

train_loop(model, optimizer, X_train_NN, y_train_NN, f_train_NN, X_test_NN, y_test_NN, f_test_NN,
           D, n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PGNNIV_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 9000.
Epoch 9000, Train loss: 5.889e+03, Test loss: 3.161e+04, MSE(e): 5.716e-04, MSE(pi1): 5.649e-03, MSE(pi2): 3.967e-04, MSE(pi3): 1.154e-03
Epoch 9100, Train loss: 2.682e+03, Test loss: 3.013e+04, MSE(e): 2.529e-04, MSE(pi1): 5.262e-03, MSE(pi2): 2.314e-04, MSE(pi3): 1.006e-03
Epoch 9200, Train loss: 2.576e+03, Test loss: 2.988e+04, MSE(e): 2.423e-04, MSE(pi1): 5.270e-03, MSE(pi2): 2.258e-04, MSE(pi3): 1.002e-03
Epoch 9300, Train loss: 2.525e+03, Test loss: 2.958e+04, MSE(e): 2.372e-04, MSE(pi1): 5.276e-03, MSE(pi2): 2.224e-04, MSE(pi3): 9.994e-04
Epoch 9400, Train loss: 2.496e+03, Test loss: 2.936e+04, MSE(e): 2.343e-04, MSE(pi1): 5.276e-03, MSE(pi2): 2.201e-04, MSE(pi3): 9.978e-04
Epoch 9500, Train loss: 2.478e+03, Test loss: 2.923e+04, MSE(e): 2.325e-04, MSE(pi1): 5.270e-03, MSE(pi2): 2.185e-04, MSE(pi3): 9.968e-04
Epoch 9600, Train loss: 2.464e+03, Test loss: 2.917e+04, MSE(e): 2.312e-04, MSE(pi1): 5.262e-03, MSE(pi2): 2.174e-04, MSE(p